# 1 Project_1 问答摘要与推理

## 1.1 问题描述

项目一是由百度AI技术生态部门提供，题目为“汽车大师问答摘要与推理”。

要求大家使用汽车大师提供的11万条（技师与用户的多轮对话与诊断建议报告数据）建立模型，模型需基于对话文本、用户问题、车型与车系，输出包含摘要与推断的报告文本，综合考验模型的归纳总结与推断能力。该解决方案可以节省大量人工时间，提高用户获取回答和解决方案的效率。

## 1.2 数据说明

对于每个用户问题"QID"，有对应文本形式的文本集合 D = "Brand", "Collection", "Problem", "Conversation"，要求阅读理解系统自动对D进行分析，输出相应的报告文本"Report"，其中包含摘要与推理。目标是"Report"可以正确、完整、简洁、清晰、连贯地对D中的信息作归纳总结与推理。

训练：所提供的训练集（82943条记录）建立模型，基于汽车品牌、车系、问题内容与问答对话的文本，输出建议报告文本

输出结果：对所提供的测试集（20000条记录）使用训练好的模型，输出建议报告的结果文件，通过最终测评得到评价分数

请提交一个CSV文件，包含QID和Prediction两个字段，分隔符为逗号(',')，请注意区分大小写。参考样例如下：


QID

Prediction


Q103432 你的预测 
Q100965 你的预测 

训练、测试数据


In [4]:
# 加载数据
import pandas as pd
train_path = "week1-homework/datasets/AutoMaster_TrainSet.csv"
test_path = "week1-homework/datasets/AutoMaster_TestSet.csv"
df = pd.read_csv(train_path, encoding='utf-8')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82943 entries, 0 to 82942
Data columns (total 6 columns):
QID         82943 non-null object
Brand       81642 non-null object
Model       81642 non-null object
Question    82943 non-null object
Dialogue    82941 non-null object
Report      82873 non-null object
dtypes: object(6)
memory usage: 3.8+ MB


In [6]:
df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


# 2、分词器介绍
分词器的不同，分出的结果也有差别，对模型训练的训练起着一定的影响,下面主要介绍jieba、SnowNlp、pkuseg、THULAC、LAC五种分词器

## 2、1  jieba
•jieba.cut 方法接受三个输入参数: 需要分词的字符串；cut_all 参数用来控制是否采用全模式；HMM 参数用来控制是否使用 HMM 模型

•jieba.cut_for_search 方法接受两个参数：需要分词的字符串；是否使用 HMM 模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细待分词的字符串可以是 unicode 或 UTF-8 字符串、GBK 字符串。注意：不建议直接输入 GBK 字符串，可能无法预料地错误解码成 UTF-8

•jieba.cut 以及 jieba.cut_for_search 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)，或者用

•jieba.lcut 以及 jieba.lcut_for_search 直接返回 list

•jieba.Tokenizer(dictionary=DEFAULT_DICT) 新建自定义分词器，可用于同时使用不同词典。jieba.dt 为默认分词器，所有全局分词相关函数都是该分词器的映射。


In [11]:
import jieba
#全模式
seg_text=jieba.cut("我来到北京清华大学",cut_all=True)
print('Full Mode:' + '/'.join(seg_text))
#精确模式
seg_text=jieba.cut("我来到北京清华大学",cut_all=False)
print('Default Mode:' + '/'.join(seg_text))
#默认模式
seg_text=jieba.cut("我来到北京清华大学")
print('Default Mode:' + '/'.join(seg_text))

#搜索引擎模式
seg_text=jieba.cut_for_search("我来到北京清华大学")
print('Search Egine Mode:' + '/'.join(seg_text))

Full Mode:我/来到/北京/清华/清华大学/华大/大学
Default Mode:我/来到/北京/清华大学
Default Mode:我/来到/北京/清华大学
Search Egine Mode:我/来到/北京/清华/华大/大学/清华大学


In [22]:
import jieba.posseg as pseg
#带词性的分割，可实现有规则的词性标注功能
# jieba.load_userdict("filename")
jieba.add_word("我",tag="自定义标注")   #利用结巴完成标注
words=pseg.cut("我来到北京清华大学")
for w,s in words:
    print("单词= ({})，词性= ({})".format(w,s))

单词= (我)，词性= (自定义标注)
单词= (来到)，词性= (v)
单词= (北京)，词性= (ns)
单词= (清华大学)，词性= (nt)


## 2、2  snownlp
SnowNLP是一个python写的类库，可以方便的处理中文文本内容，是受到了TextBlob的启发而写的，由于现在大部分的自然语言处理库基本都是针对英文的，
于是写了一个方便处理中文的类库，并且和TextBlob不同的是，这里没有用NLTK，所有的算法都是自己实现的，并且自带了一些训练好的字典。
注意本程序都是处理的unicode编码，所以使用时请自行decode成unicode。


In [23]:
from snownlp import SnowNLP 
s = SnowNLP("我来到北京清华大学")
print(' '.join(s.words))

ModuleNotFoundError: No module named 'snownlp'

## 2、3 pkuseg
pkuseg具有如下几个特点：

1.多领域分词。不同于以往的通用中文分词工具，此工具包同时致力于为不同领域的数据提供个性化的预训练模型。根据待分词文本的领域特点，用户可以自由地选择不同的模型。 我们目前支持了新闻领域，网络领域，医药领域，旅游领域，以及混合领域的分词预训练模型。在使用中，如果用户明确待分词的领域，可加载对应的模型进行分词。如果用户无法确定具体领域，推荐使用在混合领域上训练的通用模型。各领域分词样例可参考 example.txt。

2.更高的分词准确率。相比于其他的分词工具包，当使用相同的训练数据和测试数据，pkuseg可以取得更高的分词准确率。

3.支持用户自训练模型。支持用户使用全新的标注数据进行训练。

4.支持词性标注。

In [24]:
import pkuseg
pku_seg = pkuseg.pkuseg()
print(' '.join(pku_seg.cut('我来到北京清华大学')))

ModuleNotFoundError: No module named 'pkuseg'

## 2、4 THULAC
THULAC（THU Lexical Analyzer for Chinese）由清华大学自然语言处理与社会人文计算实验室研制推出的一套中文词法分析工具包，具有中文分词和词性标注功能。THULAC具有如下几个特点：

1.能力强。利用我们集成的目前世界上规模最大的人工分词和词性标注中文语料库（约含5800万字）训练而成，模型标注能力强大。

2.准确率高。该工具包在标准数据集Chinese Treebank（CTB5）上分词的F1值可达97.3％，词性标注的F1值可达到92.9％，与该数据集上最好方法效果相当。

3.速度较快。同时进行分词和词性标注速度为300KB/s，每秒可处理约15万字。只进行分词速度可达到1.3MB/s。

In [ ]:
import thulac
thu_lac = thulac.thulac(seg_only=True) 
thu_result = thu_lac.cut("我来到北京清华大学", text=True) 
print(thu_result)

## 2、5 LAC
LAC全称Lexical Analysis of Chinese，是百度自然语言处理部研发的一款联合的词法分析工具，实现中文分词、词性标注、专名识别等功能。该工具具有以下特点与优势：

- 效果好：通过深度学习模型联合学习分词、词性标注、专名识别任务，整体效果F1值超过0.91，词性标注F1值超过0.94，专名识别F1值超过0.85，效果业内领先。
- 效率高：精简模型参数，结合Paddle预测库的性能优化，CPU单线程性能达800QPS，效率业内领先。
- 可定制：实现简单可控的干预机制，精准匹配用户词典对模型进行干预。词典支持长片段形式，使得干预更为精准。
- 调用便捷：支持一键安装，同时提供了Python、Java和C++调用接口与调用示例，实现快速调用和集成。
- 支持移动端: 定制超轻量级模型，体积仅为2M，主流千元手机单线程性能达200QPS，满足大多数移动端应用的需求，同等体积量级效果业内领先。

In [27]:
from LAC import LAC
lac = LAC(mode='seg')
text = u"我来到北京清华大学"
texts=[u"LAC是个优秀的分词工具", u"百度是一家高科技公司"]
seg_result = lac.run(texts)
print(seg_result)

ModuleNotFoundError: No module named 'LAC'

# 3、 w2v训练
采用gensim来进行词向量训练

In [1]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.keyedvectors import KeyedVectors

D:\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 3、1 加载数据--》新建模型--》模型训练--》模型保存

In [36]:
sentence_path="week1-homework\datasets\sentences.txt"    #加载数据：所有语料的分词，一行一个句子的分词
# sentences=[line.strip().split() for line in sentences]   #不建议使用，占大量内存
sentences=LineSentence(sentence_path)    #返回是迭代器格式
#模型建立&训练：size=向量长度，sg=1（skip-gram）,window=窗口大小,iter=迭代次数
w2v=Word2Vec(sentences=sentences,size=256,sg=1,window=5,iter=20)   

#保存词向量
word_dict={}
for word in w2v.wv.vocab:
    word_dict[word] = w2v[word]
print(word_dict)
#--------->保存词向量方法pkl
#保存模型
#方式一：KeyedVectors实例的形式保存，不是模型的完整状态，不可再次进行训练
w2v.wv.save_word2vec_format("xxx.bin", binary=True)
#方式二：完整保存
w2v.save("xxx.model")

D:\anaconda\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
D:\anaconda\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 3、2 加载模型----》再次训练

In [37]:
sentence_path="week1-homework\datasets\sentences.txt"    #加载数据：所有语料的分词，一行一个句子的分词
# sentences=[line.strip().split() for line in sentences]   #不建议使用，占大量内存
sentences=LineSentence(sentence_path)    #返回是迭代器格式
w2v=Word2Vec.load("xxx.model")
w2v.train(sentences,total_examples=1,epochs=1)

D:\anaconda\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(13450318, 20446635)

## 3、3 加载模型--》。。。

In [11]:
model = KeyedVectors.load_word2vec_format("xxx.bin", binary=True)
word_dict = {}
for word in model.vocab:
    word_dict[word] = model[word]
word_dict   #存入文本可以转格式

D:\anaconda\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


TypeError: sequence item 0: expected str instance, numpy.float32 found

## 3、4 annoy来实现most_similar，加速

In [24]:
import time
def timeit(f):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = f(*args, **kwargs)
        end_time = time.time()
        print("%s函数运行时间为：%.8f" %(f.__name__, end_time - start_time))
        return res
    return wrapper

In [25]:
@timeit
def run():
    from collections import OrderedDict
    from annoy import AnnoyIndex
    word_index = OrderedDict()   #word id映射
    for counter, key in enumerate(model.vocab.keys()):
        word_index[key] = counter

    wv_index = AnnoyIndex(256)   #向量size
    i = 0
    for key in model.vocab.keys():    #任意词向量均可
        v = model[key]
        wv_index.add_item(i,v)    #id 向量 映射表,v为list或ndarray
        i += 1
    wv_index.build(10)  #构建树
    # wv_index.save('wv_index_build10.index')   #保存结构
    # wv_index=AnnoyIndex(256)
    # wv_index.load('')           #模型加载

    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    for item in wv_index.get_nns_by_item(word_index[u'奖赏'], 11):    #word_id,向量的11个最相似
        print(reverse_word_index[item])
run()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


奖赏
客气"
886
好勒
太棒了
明儿
随时欢迎
生意兴隆
祝你好运
大吉
夸奖
run函数运行时间为：1.27857852
